## Converts .eeg file to pandas dataframe

In [6]:
import numpy as np
import struct

def eeg_to_ascii(file_name, chanlist='all', triallist='all', typerange='all', accepttype='all', rtrange='all', responsetype='all', data_format='auto'):
    with open(file_name, 'rb') as f:
        # Read general part of the ERP header and set variables
        f.read(20) # skip revision number
        f.read(342) # skip the first 362 bytes

        nsweeps = struct.unpack('<H', f.read(2))[0] # number of sweeps
        f.read(4) # skip 4 bytes
        pnts = struct.unpack('<H', f.read(2))[0] # number of points per waveform
        chan = struct.unpack('<H', f.read(2))[0] # number of channels
        f.read(4) # skip 4 bytes
        rate = struct.unpack('<H', f.read(2))[0] # sample rate (Hz)
        f.read(127) # skip 127 bytes
        xmin = struct.unpack('<f', f.read(4))[0] # in s
        xmax = struct.unpack('<f', f.read(4))[0] # in s
        f.read(387) # skip 387 bytes

        # Read electrode configuration
        chan_names = []
        baselines = []
        sensitivities = []
        calibs = []
        factors = []
        for elec in range(chan):
            chan_name = f.read(10).decode('ascii').strip('\x00')
            chan_names.append(chan_name)
            f.read(37) # skip 37 bytes
            baseline = struct.unpack('<H', f.read(2))[0]
            baselines.append(baseline)
            f.read(10) # skip 10 bytes
            sensitivity = struct.unpack('<f', f.read(4))[0]
            sensitivities.append(sensitivity)
            f.read(8) # skip 8 bytes
            calib = struct.unpack('<f', f.read(4))[0]
            calibs.append(calib)
            factor = calib * sensitivity / 204.8
            factors.append(factor)

    # Read and process epoch datapoints data
    data = np.empty((nsweeps, len(chan_names), pnts), dtype=float)
    sweep_headers = []

    # Constants for the sweep header size in bytes and data point size in bytes
    SWEEP_HEAD_SIZE = 13
    DATA_POINT_SIZE = 4

    with open(file_name, 'rb') as f:
        # Ensure the file pointer is at the beginning of the EEG data
        f.seek((900 + chan * 75))

        for sweep in range(nsweeps):
            # Read the sweep header
            f.read(SWEEP_HEAD_SIZE)
            # ttype = struct.unpack('<h', f.read(2))[0]
            # accept = f.read(1)
            # correct = struct.unpack('<h', f.read(2))[0]
            # rt = struct.unpack('<f', f.read(4))[0]
            # response = struct.unpack('<h', f.read(2))[0]
            # #reserved  struct.unpack('<h', f.read(2))[0]
            # sweep_headers.append((ttype, accept, correct, rt, response))

            for point in range(pnts):
                for channel in range(chan):

                    # Read the data point as a 2-byte integer
                    value = struct.unpack('<l', f.read(DATA_POINT_SIZE))[0]

                    # Scale the data point to microvolts and store it in the data array
                    data[sweep, channel, point] = value * factors[channel]

    # Return relevant data in ASCII format (or any other format you'd like)
    return data, chan_names

file_name = "/home/woess/current assignment/read_neuro_scratch/16017_rest_ec.eeg"


eeg_data, channels = eeg_to_ascii(file_name)
print("Channels:", channels)
print("EEG data:", eeg_data)

Channels: ['O2', 'O1', 'Pz', 'P4', 'P8', 'C4', 'T8', 'P7', 'P3', 'Cz', 'C3', 'Fz', 'F4', 'F8', 'T7', 'F3', 'Fp2', 'F7', 'Fp1', 'FT7', 'FC3', 'FCz', 'FC4', 'FT8', 'TP7', 'CP3', 'CPz', 'CP4', 'TP8', 'Oz', 'M2', 'EKG', 'HEOG', 'VEOG', 'Startle']
EEG data: [[[-1.03507472e+00 -1.25362509e+00 -1.58189545e+00 ...  1.01407965e+01
    8.57847270e+00  7.81176716e+00]
  [ 2.87407081e+00  3.38130338e+00  4.04318185e+00 ...  1.28498425e+01
    1.03706154e+01  8.09051526e+00]
  [ 6.74066277e+00  5.77097524e+00  4.74909658e+00 ... -1.01304176e+01
   -1.03525264e+01 -9.99222967e+00]
  ...
  [ 4.14800892e+00  3.89417023e+00  3.63603170e+00 ...  4.58451658e+00
    4.54774556e+00  4.55174885e+00]
  [ 1.13392650e+01  1.13626917e+01  1.13747016e+01 ... -3.82765619e+01
   -3.81490495e+01 -3.79926243e+01]
  [-3.25453231e-01 -2.26556964e-01 -6.07908085e-02 ... -1.73476210e-01
   -3.86392310e-01 -5.17611494e-01]]

 [[ 7.89465024e+00  8.18941153e+00  8.21135552e+00 ... -1.90112133e+00
   -3.50169884e+00 -6.0713

In [7]:
#Place eeg_data into a pandas dataframe
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(eeg_data[0].transpose(), columns=channels)
#Display all decimals
pd.set_option('display.float_format', lambda x: '%.6f' % x)
df.head()

#plot the data

# plt.plot(df.index.to_list(),df[0].to_list())
# #Restrict x axis to 0-100
# plt.xlim(0,250)
# plt.show()


,O2,O1,Pz,P4,P8,C4,T8,P7,P3,Cz,...,CP3,CPz,CP4,TP8,Oz,M2,EKG,HEOG,VEOG,Startle
0,-1.035075,2.874071,6.740663,0.154794,-2.462176,3.829969,25.461563,1.892966,10.256744,10.105953,...,15.309201,8.894288,2.736179,2.539128,1.379210,-0.449111,-1.915504,4.148009,11.339265,-0.325453
1,-1.253625,3.381303,5.770975,-0.190527,-2.684730,4.264401,16.572167,2.724614,10.140648,9.972658,...,15.456137,8.399362,2.388634,1.399523,1.457348,-0.694794,-1.936854,3.894170,11.362692,-0.226557
2,-1.581895,4.043182,4.749097,-0.799770,-2.944796,4.701057,6.485786,3.863923,9.928918,9.783317,...,15.763650,7.939131,2.000018,0.168732,1.604877,0.119951,-1.974663,3.636032,11.374702,-0.060791
3,-1.753296,4.977581,3.847613,-1.457348,-3.029013,4.907004,-1.561138,5.173298,9.837436,9.588638,...,16.145149,7.518489,1.651138,-0.859523,1.943230,1.284317,-2.040051,3.377745,11.377074,0.010082
4,-1.623115,6.267532,3.410216,-1.637349,-2.657744,5.370201,-5.852079,6.641025,10.069627,9.722378,...,16.687225,7.421965,1.679309,-1.494268,2.453725,0.994893,-2.072522,3.123906,11.371884,0.033361
